# Add Forecasts to `System`

**Originally Contributed by**: Clayton Barrows

## Introduction

An example of how to parse add time series data to a `System` using [PowerSystems.jl](github.com/NREL-SIIP/PowerSystems.jl)

For example, a `System` created by [parsing a MATPOWER file](../../notebook/PowerSystems_examples/parse_matpower.ipynb)
doesn't contain any time series data. So a user may want to add forecasts to the `System`
### Dependencies
Let's use the 5-bus dataset we parsed in the MATPOWER example

In [1]:
using SIIPExamples
pkgpath = dirname(dirname(pathof(SIIPExamples)))
include(joinpath(pkgpath, "test", "PowerSystems_examples", "parse_matpower.jl"))

Project SIIPExamples v0.0.2
Status `~/Documents/GitRepos/SIIPExamples.jl/Project.toml`
  [69666777] Arrow v0.4.1
  [336ed68f] CSV v0.8.0
  [9961bab8] Cbc v0.7.1
  [41994980] D3TypeTrees v0.1.1
  [a93c6f00] DataFrames v0.22.0
  [60bf3e95] GLPK v0.14.2
  [2cd47ed4] InfrastructureSystems v1.0.4
  [b6b21f68] Ipopt v0.6.3
  [2535ab7d] JSON2 v0.3.2
  [4076af6c] JuMP v0.21.5
  [98b081ad] Literate v2.7.0
  [47be7bcc] ORCA v0.5.0
  [f0f68f2c] PlotlyJS v0.14.0
  [91a5bcdd] Plots v1.9.1
  [5f7eddb3] PowerGraphics v0.6.1
  [e690365d] PowerSimulations v0.8.0
  [bcd98974] PowerSystems v1.0.2
  [9e3dc215] TimeSeries v0.19.1
  [f269a46b] TimeZones v1.5.0
  [0f1e0344] WebIO v0.8.15
  [ade2ca70] Dates
  [56ddb016] Logging
  [44cfe95a] Pkg
  [9a3f8284] Random
  [10745b16] Statistics
  [8dfed614] Test
┌ Info: extending matpower format with data: areas 1x3
└ @ PowerSystems /Users/sdalvi/.julia/packages/PowerSystems/eF3Pv/src/parsers/pm_io/matpower.jl:332
┌ Info: extending matpower format with data: gen_nam

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,6
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
4,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
5,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
8,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
9,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5


### Define pointers to time series files
For example, if we want to add a bunch of time series files, say one for each load and
one for each renewable generator, we need to define pointers to each .csv file containing
the time series in the following format (PowerSystems.jl also supports a CSV format for this file)

In [2]:
FORECASTS_DIR = joinpath(base_dir, "forecasts", "5bus_ts")
fname = joinpath(FORECASTS_DIR, "timeseries_pointers_da.json")
open(fname, "r") do f
    for line in eachline(f)
        println(line)
    end
end

[
    {
        "simulation": "DAY_AHEAD",
        "resolution": 3600,
        "category": "Generator",
        "component_name": "SolarBusC",
        "module": "InfrastructureSystems",
        "type": "SingleTimeSeries",
        "name": "max_active_power",
        "scaling_factor_multiplier": "get_max_active_power",
        "scaling_factor_multiplier_module": "PowerSystems",
        "normalization_factor": 1.0,
        "data_file": "./gen/Renewable/PV/da_solar5.csv"
    },
    {
        "simulation": "DAY_AHEAD",
        "resolution": 3600,
        "category": "Generator",
        "component_name": "WindBusA",
        "module": "InfrastructureSystems",
        "type": "SingleTimeSeries",
        "name": "max_active_power",
        "scaling_factor_multiplier": "get_max_active_power",
        "scaling_factor_multiplier_module": "PowerSystems",
        "normalization_factor": 1.0,
        "data_file": "./gen/Renewable/WIND/da_wind5.csv"
    },
    {
        "simulation": "DAY_AHEAD",
   

### Read the pointers

In [3]:
ts_pointers = PowerSystems.IS.read_time_series_file_metadata(fname)

5-element Array{InfrastructureSystems.TimeSeriesFileMetadata,1}:
 InfrastructureSystems.TimeSeriesFileMetadata("DAY_AHEAD", "Generator", "SolarBusC", "max_active_power", 1.0, "/Users/sdalvi/.julia/packages/PowerSystems/eF3Pv/data/forecasts/5bus_ts/gen/Renewable/PV/da_solar5.csv", Millisecond(3600000), Float64[], SingleTimeSeries, nothing, "get_max_active_power", "PowerSystems")
 InfrastructureSystems.TimeSeriesFileMetadata("DAY_AHEAD", "Generator", "WindBusA", "max_active_power", 1.0, "/Users/sdalvi/.julia/packages/PowerSystems/eF3Pv/data/forecasts/5bus_ts/gen/Renewable/WIND/da_wind5.csv", Millisecond(3600000), Float64[], SingleTimeSeries, nothing, "get_max_active_power", "PowerSystems")
 InfrastructureSystems.TimeSeriesFileMetadata("DAY_AHEAD", "ElectricLoad", "bus2", "max_active_power", 1.0, "/Users/sdalvi/.julia/packages/PowerSystems/eF3Pv/data/forecasts/5bus_ts/load/da_load5.csv", Millisecond(3600000), Float64[], SingleTimeSeries, nothing, "get_max_active_power", "PowerSystems")
 I

### Read and assign forecasts to `System` using the `ts_pointers` struct

In [4]:
add_time_series!(sys, ts_pointers)
sys

,ConcreteType,SuperTypes,Count
,String,String,Int64
1,Arc,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,6
2,Area,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
3,Bus,Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
4,Line,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5
5,LoadZone,AggregationTopology <: Topology <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,1
6,PhaseShiftingTransformer,ACBranch <: Branch <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
7,PowerLoad,StaticLoad <: ElectricLoad <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,3
8,RenewableDispatch,RenewableGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,2
9,ThermalStandard,ThermalGen <: Generator <: StaticInjection <: Device <: Component <: InfrastructureSystemsComponent <: InfrastructureSystemsType <: Any,5


---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*